In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy ADK Agent in AI Engine and Agentspace

This notebook provides a step-by-step to deploy the Data Assistant ADK agent on Vertex AI agent engine, and deploy to Agentspace

- Import Libraries and Setup env_vars
- Package and Deploy Agent on Agent Engine
- Deploy on Agentspace

### Setup and Config

In [ ]:
# Basic Libraries
import os 
import vertexai

# AI Engine on Vertex AI 
from vertexai import agent_engines

# Library for AI Engine with ADK
from vertexai.preview import reasoning_engines

# To view JSON response formatted 
import json
from IPython.display import JSON


In [ ]:
# Load environment variables from data_assistant .env file 

from dotenv import load_dotenv
load_dotenv('./data_assistant/.env')
env_vars = {}

In [ ]:
# Load Variables on env_vars dict to be used when creating the Agent

env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["GOOGLE_CLOUD_PROJECT"] = os.getenv("GOOGLE_CLOUD_PROJECT")
env_vars["GOOGLE_CLOUD_LOCATION"] = os.getenv("GOOGLE_CLOUD_LOCATION")
env_vars["GOOGLE_CLOUD_STORAGE_BUCKET"] = os.getenv("GOOGLE_CLOUD_STORAGE_BUCKET")


env_vars["BQ_PROJECT_ID"] = os.getenv("BQ_PROJECT_ID")
env_vars["BQ_DATASET_ID"] = os.getenv("BQ_DATASET_ID")
env_vars["BQ_ROOT_MODEL"] = os.getenv("BQ_ROOT_MODEL")
env_vars["BQ_TOOL_MODEL"] = os.getenv("BQ_TOOL_MODEL")
env_vars["BQ_LOCATION"] = os.getenv("BQ_LOCATION")
env_vars["BQ_METADATA_MODE"] = os.getenv("BQ_METADATA_MODE")
env_vars

In [ ]:
# Instantiate Vertex AI

vertexai.init(
    project=env_vars["GOOGLE_CLOUD_PROJECT"],
    location=env_vars["GOOGLE_CLOUD_LOCATION"],
    staging_bucket=env_vars["GOOGLE_CLOUD_STORAGE_BUCKET"],
)

### Agent ADK Package and Deploy

In [ ]:
# Importong root_agent from local data_assistant folder
from data_assistant.agent import root_agent

In [ ]:
# Requirements file
requirements = [
    "google-cloud-aiplatform[agent_engines,adk]",
    "cloudpickle",
    "pydantic",
    "google-cloud-bigquery",
    "pandas",
    "db-dtypes"
]

In [ ]:
# Extra packages from foder data_assistant
extra_packages = ["data_assistant"]

In [ ]:
# Agent Metadata 

agent_display_name = "BQ Data Assistant"

agent_description = """ Helps the user get data and metadata on BigQuery Dataset """

In [ ]:
# Instantiate the Assistant as an ADK App 
app = reasoning_engines.AdkApp(
    agent=root_agent,
)

In [ ]:
# For testing purposes create a session
session = app.create_session(user_id="user_1")
session

In [ ]:
# Run a simple query
for event in app.stream_query(
    user_id="user_1",
    session_id=session.id,
    message="Which tables exist in this dataset?",
):
    print(event)

In [ ]:
json_answer = event['content']['parts'][0]['text']
json_answer = json_answer.replace('```json','').replace('```','').replace('\n','').replace('\\n','')

JSON(json.loads(json_answer))

In [ ]:
## (OPTIONAL) - Retrieve all existent Agent Engine resource.names (Agents)
for agent in agent_engines.list():
    print(f"Agent: {agent.display_name} [{agent.resource_name}] created/updated at: {agent.update_time}" )

In [ ]:
## (OPTIONAL) - Update the agent at the same Agent Resource 
# agent_engines.update(
#     resource_name=remote_agent.resource_name,    # Required.
#     agent_engine = app,
#     extra_packages=extra_packages,      # Extra packages
#     display_name=agent_display_name,    # Display name  
#     description=agent_description,      # Description
#     env_vars=env_vars 
# )

In [ ]:
# Deploy the Agent on AI Engine (This takes a few minutes)

remote_agent = agent_engines.create(
    app,                                # The Agente instantiated as ADK agent
    requirements=requirements,          # Requirements file
    extra_packages=extra_packages,      # Extra packages
    display_name=agent_display_name,    # Display name  
    description=agent_description,      # Description
    env_vars=env_vars                   # Env Vars dict
)

In [ ]:
# Retrieve ID of Agent Engine
remote_agent.resource_name

In [ ]:
# Testing Remote Agent
session_remote = remote_agent.create_session(user_id="user_1")
session_remote['id']

In [ ]:
# Query on remote agent for test 
for event in remote_agent.stream_query(
    user_id="user_1",
    session_id=session_remote['id'],  
    message="Which tables exist in this dataset?",
):
   print(event)

In [ ]:
# To view JSON response formatted 
json_answer = event['content']['parts'][0]['text']
json_answer = json_answer.replace('```json','').replace('```','').replace('\n','').replace('\\n','')

JSON(json.loads(json_answer))

### Deploy Agent on Agentspace

This is a temporary step, due the official way to do this still working in progress

In [ ]:
import subprocess
import json
import requests

# The ID of your Google Cloud project.
project_id = env_vars["GOOGLE_CLOUD_PROJECT"] 

# The ID of the Agentspace app (see: https://cloud.google.com/agentspace/agentspace-enterprise/docs/assistant ).
app_id = ""

# The display name of the agent.
display_name = "BQ Data Assistant"

# The description of the agent, displayed on the frontend; it is only for the user’s benefit.
description = "Useful agent to help you find information on BigQuery"

# The description of the agent used by the LLM to route requests to the agent.
# Must properly describe what the agent does. Never shown to the user.
tool_description = "The agent can Help the user get data and metadata on BigQuery Dataset"

# The ID of the reasoning engine endpoint where the ADK agent is deployed (Resource.name).
adk_deployment_name = "" # The remote_agent.resource_name Deployed
 

In [ ]:
# Get the access token from gcloud
try:
    access_token = subprocess.check_output(
        "gcloud auth print-access-token", shell=True, text=True
    ).strip()
except subprocess.CalledProcessError as e:
    print(f"Error getting access token: {e}")
    exit()

# API endpoint
url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant/agents"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": project_id,
}

# Data payload
data = {
    "displayName": display_name,
    "description": description,
    "adk_agent_definition": {
        "tool_settings": {"tool_description": tool_description},
        "provisioned_reasoning_engine": {"reasoning_engine": adk_deployment_name},
    },
}

In [ ]:
# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response JSON: \n")
JSON(response.json())